# 对tool use进行微调

> `tools`是Chat Completion API中的一个可选参数，可用于提供函数规范。其目的是使模型能够生成符合所提供规范的函数参数。请注意，API实际上并不会执行任何函数调用。开发者需要使用模型输出来执行函数调用。

当函数调用按预期工作时，它是一个非常强大的工具。然而，我们发现，随着函数数量的增加，以及任务复杂度的提高，函数调用的准确性会降低（例如：更多的虚构调用和错误调用）。

在为函数调用进行微调之前，最好先：

1. 改进函数定义: 使函数定义更清晰,相互之间更容易区分
2. 尝试提示词工程: 通常更详细的提示可以帮助模型调用正确的函数

_如果_ 上述步骤无法将函数调用提升到令人满意的水平,那么可以尝试对函数调用进行微调。

### 概述


- **评估基线函数调用性能**：在我们给定的函数上评估开箱即用的`gpt-4o-mini`模型（假设出于延迟和成本原因，我们无法为无人机副驾驶使用`gpt-4o`）。
- **生成合成数据**：（重点）问答对先做回答在做问题。排列组合所有可能的函数调用的结果，入参具体的值是枚举的直接枚举，不可枚举的用gpt 4o生成参数值。这些作为问答对的回答。最后使用`gpt-4o`为每个tool use每个可能的回答创建2个问题。
- **微调**：运行微调作业，并评估微调后的模型。

注意: _本教程展示了如何仅使用函数列表来创建用于函数调用微调的合成训练数据。虽然实际生产环境的测试评估更可取,但这种方法也能产生很好的结果,并且可以与实际训练数据结合使用。_

## 安装必要的包

首先,我们需要安装一些必要的 Python 包:

In [ ]:
# !pip install tenacity -q
# !pip install openai -q
# !pip install typing -q
# !pip install python-dotenv

## 初始化设置

导入所需的库并设置 OpenAI 客户端:

In [3]:
import numpy as np
import json
import os
from IPython.display import display
import pandas as pd
import openai
from openai import OpenAI
import itertools
import time
import base64

from openai.types import CompletionUsage
from openai.types.chat import ChatCompletionMessage
from tenacity import retry, wait_random_exponential, stop_after_attempt
from typing import Any, Dict, List, Generator, Tuple
import ast


client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

### 工具函数

让我们定义一些实用函数，用于调用Chat Completions API，一个用于获取完成结果，一个用于获取函数调用。

获取完成结果的函数比较简单 就是发送问话，提取回答，我们主要看看评估模型性能的函数，主要用于测试模型选择正确函数的能力是怎么构建的：


1. **初始化阶段**
   - 创建空列表存储实际结果(prompts_to_actual)、延迟时间(latencies)和token使用量(tokens_used)

2. **评估循环**
   - 对每个测试样例进行测试：
     - 构建消息列表(包含系统提示和用户输入)
     - 记录开始时间
     - 调用模型获取结果(get_chat_completion)
     - 计算并记录:
       - 延迟时间(毫秒)
       - 实际选择的函数名
       - 使用的token数量

3. **性能统计计算**
   - 计算正确匹配的数量和百分比（比较实际选择的函数(prompts_to_actual)与期望函数(prompts_to_expected_tool_name)是否一致）
   - 计算平均延迟时间
   - 计算平均token使用量

4. **结果可视化**
   - 创建DataFrame存储详细结果
   - 为每行添加样式(匹配失败显示红色背景)
   - 显示结果表格
   - 打印总体统计信息（匹配率、平均延迟、平均token使用量）

这段代码的主要目的是：
1. 测试模型是否能正确选择函数
2. 评估模型的性能指标（准确率、延迟、资源使用）
3. 以直观的方式展示评估结果


In [20]:
def get_chat_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4o-mini",
    max_tokens=500,
    temperature=0.0,
    stop=None,
    tools=None,
    seed=42,
    functions=None,
    tool_choice=None,
) -> tuple[Any, Any]:
    """
    调用 OpenAI API 获取聊天补全
    
    参数:
    messages: 聊天消息列表
    model: 使用的模型名称
    max_tokens: 最大生成令牌数
    temperature: 生成的随机性(0-1)
    stop: 停止生成的标记
    tools: 工具列表
    seed: 随机种子
    functions: 函数列表
    tool_choice: 工具选择
    
    返回:
    补全结果和使用情况统计
    """
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "tools": tools,
        "seed": seed,
        "tool_choice": tool_choice,
    }
    if functions:
        params["functions"] = functions

    completion = client.chat.completions.create(**params)
    return completion.choices[0].message, completion.usage


def eval(model: str, system_prompt: str, function_list, prompts_to_expected_tool_name):
    """
    评估模型在根据给定提示选择正确函数方面的性能。

    参数：
        model (str): 要评估的模型名称。
        system_prompt (str): 在聊天完成中使用的系统提示。
        function_list (list): 模型可以调用的函数列表。
        prompts_to_expected_tool_name (dict): 将提示映射到预期函数名称的字典。

    返回：
        None
    """

    prompts_to_actual = []
    latencies = []
    tokens_used = []

    for prompt, expected_function in prompts_to_expected_tool_name.items():
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ]

        start_time = time.time()
        completion, usage = get_chat_completion(
            model=model,
            messages=messages,
            seed=42,
            tools=function_list,
            temperature=0.0,
            tool_choice="required",
        )
        end_time = time.time()

        latency = (end_time - start_time) * 1000  # 转换为毫秒
        latencies.append(latency)

        prompts_to_actual.append(
            {prompt: completion.tool_calls[0].function.name})

        # 计算使用的tokens
        tokens_used.append(usage.total_tokens)

    total_prompts = len(prompts_to_expected_tool_name)

    # 计算匹配的数量
    matches = sum(
        1
        for result in prompts_to_actual
        if list(result.values())[0]
        == prompts_to_expected_tool_name[list(result.keys())[0]]
    )
    match_percentage = (matches / total_prompts) * 100

    # 计算平均延迟
    avg_latency = sum(latencies) / total_prompts
    # 计算平均使用的tokens数量
    avg_tokens_used = sum(tokens_used) / total_prompts

    # 创建一个DataFrame来存储结果
    results_df = pd.DataFrame(columns=["Prompt", "Expected", "Match"])

    results_list = []
    for result in prompts_to_actual:
        prompt = list(result.keys())[0]
        actual_function = list(result.values())[0]
        expected_function = prompts_to_expected_tool_name[prompt]
        match = actual_function == expected_function
        results_list.append(
            {
                "Prompt": prompt,
                "Actual": actual_function,
                "Expected": expected_function,
                "Match": "Yes" if match else "No",
            }
        )
    results_df = pd.DataFrame(results_list)
    from IPython.display import display
    
    # Your existing code
    def style_rows(row):
        match = row["Match"]
        background_color = "red" if match == "No" else "white"
        return [f"background-color: {background_color}; color: black"] * len(row)
    
    styled_results_df = results_df.style.apply(style_rows, axis=1)
    
    # Display the styled DataFrame
    display(styled_results_df)


    print(
        f"匹配数量: {matches} / {total_prompts} ({match_percentage:.2f}%)"
    )
    print(f"每次请求的平均延迟: {avg_latency:.2f} ms")
    print(f"每次请求的平均tokens使用量: {avg_tokens_used:.2f}")

### 基线测试

让我们构建一个智能无人机副驾驶。我们希望能够给副驾驶下达命令，让它要么调用相应的函数执行命令，要么在命令不可行时拒绝请求。

我们可以先为副驾驶定义一个系统提示。

In [5]:
DRONE_SYSTEM_PROMPT = """你是一个控制无人机的智能AI。根据用户的命令或请求，
调用你的一个函数来完成请求。如果你的可用函数无法完成请求，调用reject_request函数。
如果请求含糊不清或不明确，拒绝请求。"""

现在，让我们为副驾驶可以执行的所有操作定义函数。

In [8]:
function_list = [
    {
        "type": "function",
        "function": {
            "name": "takeoff_drone",
            "description": "启动无人机的起飞序列。",
            "parameters": {
                "type": "object",
                "properties": {
                    "altitude": {
                        "type": "integer",
                        "description": "指定无人机应上升的高度（米）。",
                    }
                },
                "required": ["altitude"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "land_drone",
            "description": "将无人机降落在当前位置或指定的着陆点。",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "enum": ["current", "home_base", "custom"],
                        "description": "指定无人机的着陆位置。",
                    },
                    "coordinates": {
                        "type": "object",
                        "description": "自定义着陆地点的GPS坐标。如果location为'custom'，则需要提供。",
                    },
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "control_drone_movement",
            "description": "指挥无人机朝特定方向移动。",
            "parameters": {
                "type": "object",
                "properties": {
                    "direction": {
                        "type": "string",
                        "enum": ["forward", "backward", "left", "right", "up", "down"],
                        "description": "无人机应移动的方向。",
                    },
                    "distance": {
                        "type": "integer",
                        "description": "无人机在指定方向上应移动的距离（米）。",
                    },
                },
                "required": ["direction", "distance"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_drone_speed",
            "description": "调整无人机的速度。",
            "parameters": {
                "type": "object",
                "properties": {
                    "speed": {
                        "type": "integer",
                        "description": "指定速度，单位为km/h。有效范围为0到100。",
                        "minimum": 0,
                    }
                },
                "required": ["speed"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "control_camera",
            "description": "控制无人机的摄像头拍摄图像或视频。",
            "parameters": {
                "type": "object",
                "properties": {
                    "mode": {
                        "type": "string",
                        "enum": ["photo", "video", "panorama"],
                        "description": "摄像头的拍摄模式。",
                    },
                    "duration": {
                        "type": "integer",
                        "description": "视频拍摄的持续时间（秒）。如果模式是'video'，则需要提供。",
                    },
                },
                "required": ["mode"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "control_gimbal",
            "description": "调整无人机的云台以稳定和控制摄像头方向。",
            "parameters": {
                "type": "object",
                "properties": {
                    "tilt": {
                        "type": "integer",
                        "description": "云台的倾斜角度（度）。",
                    },
                    "pan": {
                        "type": "integer",
                        "description": "云台的平移角度（度）。",
                    },
                },
                "required": ["tilt", "pan"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_drone_lighting",
            "description": "控制无人机的照明以提高可见性和信号传递。",
            "parameters": {
                "type": "object",
                "properties": {
                    "mode": {
                        "type": "string",
                        "enum": ["on", "off", "blink", "sos"],
                        "description": "无人机的照明模式。",
                    }
                },
                "required": ["mode"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "return_to_home",
            "description": "命令无人机返回其家或起飞位置。",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_battery_saver_mode",
            "description": "切换省电模式。",
            "parameters": {
                "type": "object",
                "properties": {
                    "status": {
                        "type": "string",
                        "enum": ["on", "off"],
                        "description": "切换省电模式。",
                    }
                },
                "required": ["status"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_obstacle_avoidance",
            "description": "配置避障设置。",
            "parameters": {
                "type": "object",
                "properties": {
                    "mode": {
                        "type": "string",
                        "enum": ["on", "off"],
                        "description": "切换避障功能。",
                    }
                },
                "required": ["mode"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_follow_me_mode",
            "description": "启用或禁用“跟随我”模式。",
            "parameters": {
                "type": "object",
                "properties": {
                    "status": {
                        "type": "string",
                        "enum": ["on", "off"],
                        "description": "切换“跟随我”模式。",
                    }
                },
                "required": ["status"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calibrate_sensors",
            "description": "启动无人机传感器的校准序列。",
            "parameters": {"type": "object", "properties": {}},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_autopilot",
            "description": "启用或禁用自动驾驶模式。",
            "parameters": {
                "type": "object",
                "properties": {
                    "status": {
                        "type": "string",
                        "enum": ["on", "off"],
                        "description": "切换自动驾驶模式。",
                    }
                },
                "required": ["status"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "configure_led_display",
            "description": "配置无人机的LED显示模式和颜色。",
            "parameters": {
                "type": "object",
                "properties": {
                    "pattern": {
                        "type": "string",
                        "enum": ["solid", "blink", "pulse", "rainbow"],
                        "description": "LED显示的模式。",
                    },
                    "color": {
                        "type": "string",
                        "enum": ["red", "blue", "green", "yellow", "white"],
                        "description": "LED显示的颜色。如果模式为'rainbow'，则不需要提供。",
                    },
                },
                "required": ["pattern"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_home_location",
            "description": "设置或更改无人机的归巢位置。",
            "parameters": {
                "type": "object",
                "properties": {
                    "coordinates": {
                        "type": "object",
                        "description": "归巢位置的GPS坐标。",
                    }
                },
                "required": ["coordinates"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "reject_request",
            "description": "如果请求不可行，使用此函数。",
            "parameters": {"type": "object", "properties": {}},
        },
    },
]

首先，让我们看看函数调用在一些直接可行的提示下的表现，然后再尝试一些明显不可能的请求，这些请求应该调用`reject_request`函数。

In [9]:
straightforward_prompts_to_expected = {
    "让无人机在基地着陆": "land_drone",
    "让无人机起飞到50米高度": "takeoff_drone",
    "将速度改为每小时15公里": "set_drone_speed",
    "变成一头大象！": "reject_request",
    "让无人机向前移动10米": "control_drone_movement",
    "我想让LED显示屏以红色闪烁": "configure_led_display",
    "你能拍张照片吗？": "control_camera",
    "你能检测障碍物吗？": "set_obstacle_avoidance",
    "你能为我跳舞吗？": "reject_request",
    "你能跟随我吗？": "set_follow_me_mode",
}

In [10]:
# 使用给定的提示评估模型
eval(
    model="gpt-4o-mini",
    system_prompt=DRONE_SYSTEM_PROMPT,
    function_list=function_list,
    prompts_to_expected_tool_name=straightforward_prompts_to_expected,
)

匹配数量: 9 / 10 (90.00%)
每次请求的平均延迟: 1179.62 ms
每次请求的平均tokens使用量: 827.00


很好！模型在这些请求上表现得很好。现在让我们尝试一些更困难的请求：这些请求看起来几乎可行，且与无人机相关，但实际上无人机无法执行，飞行员应该拒绝。

In [86]:
challenging_prompts_to_expected = {
    "播放预录音频信息": "reject_request",
    "当检测到人脸时自动拍照": "reject_request",
    "检测附近的无人机": "reject_request",
    "测量风阻": "reject_request",
    "拍摄慢动作视频": "reject_request",
    "让无人机同时向前和向后移动相同的距离。": "reject_request",
    "根据地面高度变化调整无人机的高度": "reject_request",
    "在LED显示屏上显示自定义信息": "reject_request",
    "将无人机的时间与智能手机同步": "reject_request",
    "当无人机飞出指定区域时发出警报": "reject_request",
}

In [85]:
# 使用具有挑战性的提示评估模型
eval(
    model="gpt-4o-mini",
    function_list=function_list,
    system_prompt=DRONE_SYSTEM_PROMPT,
    prompts_to_expected_tool_name=challenging_prompts_to_expected,
)

匹配数量: 7 / 10 (70.00%)
每次请求的平均延迟: 1468.21 ms
每次请求的平均tokens使用量: 463.20


现在我们遇到了一些问题。

在这里，模型应该拒绝所有这些请求，因为根据给定的函数，它们是不可能的、冲突的或含糊的。然而，模型却调用了一些与请求有些相关但不正确的函数。例如，当被要求“检测附近的无人机”时，模型可能调用了与之相关但不正确的函数。

在这种简单的情况下，更多的提示工程可能会解决其中的一些问题，但为了本例的目的，我们将展示如何使用微调来提高性能。此外，随着函数数量和复杂性的增加，微调变得越来越有影响力。

再次强调，我们的目标是提高性能并使用更少的tokens，因此微调使我们能够：

- **省略函数和参数描述**：从函数和参数中删除`description`字段。
- **省略参数**：从`parameters`对象中删除整个`properties`字段。
- **完全省略函数**：从`functions`数组中删除整个`function`对象。

# 生成合成数据

### 辅助函数

我们想生成每个函数的每个调用，以便我们对所有可能的调用都有全面覆盖，然后，我们将使用`gpt-4o`来提出会调用每个调用的提示，我们将使用该提示-函数调用对作为训练数据。

对于具有固定枚举的函数，生成每个调用更为简单，但对于像`control_gimbal`这样的函数，我们需要设置`tilt`和`pan`的整数值，因此要生成这些合成调用，我们将首先设置一个占位符，然后稍后使用`gpt-4o`来提出合理的值。

In [13]:
placeholder_int = "fill_in_int" # 整数占位符
placeholder_string = "fill_in_string" # 字符串占位符

下面的函数将获取`function_list`中的所有函数，并查看给定每个函数参数的所有可能调用。函数还考虑了`required`参数，以确保所有调用都是可行的。

In [17]:
def generate_permutations(
    params: Dict[str, Dict[str, Any]]
) -> Generator[Dict[str, Any], None, None]:
    """
    生成给定参数的所有可能排列。

    :param params: 包含必需和可选字段的参数字典。
    :return: 一个生成器，逐个产生每个排列。
    """

    # 从参数中提取必需字段
    required_fields = params.get("required", [])

    # 为必需字段生成排列
    required_permutations = generate_required_permutations(params, required_fields)

    # 基于每个必需排列生成可选排列
    for required_perm in required_permutations:
        yield from generate_optional_permutations(params, required_perm)


def generate_required_permutations(
    params: Dict[str, Dict[str, Any]], required_fields: List[str]
) -> List[Dict[str, Any]]:
    """
    为必需字段生成排列。

    :param params: 参数字典。
    :param required_fields: 必需字段列表。
    :return: 必需字段排列的列表。
    """

    # 获取每个必需字段的所有可能值
    required_values = [get_possible_values(params, field) for field in required_fields]

    # 从可能的值生成排列
    return [
        dict(zip(required_fields, values))
        for values in itertools.product(*required_values)
    ]


def generate_optional_permutations(
    params: Dict[str, Dict[str, Any]], base_perm: Dict[str, Any]
) -> Generator[Dict[str, Any], None, None]:
    """
    基于基本排列生成可选字段的排列。

    :param params: 参数字典。
    :param base_perm: 基本排列字典。
    :return: 一个生成器，逐个产生可选字段的每个排列。
    """

    # 通过从所有属性中减去基本排列的字段来确定可选字段
    optional_fields = set(params["properties"]) - set(base_perm)

    # 遍历可选字段的所有组合
    for field_subset in itertools.chain.from_iterable(
        itertools.combinations(optional_fields, r)
        for r in range(len(optional_fields) + 1)
    ):

        # 为当前字段子集生成可能值的乘积
        for values in itertools.product(
            *(get_possible_values(params, field) for field in field_subset)
        ):

            # 通过组合基本排列和当前字段值创建新排列
            new_perm = {**base_perm, **dict(zip(field_subset, values))}

            yield new_perm


def get_possible_values(params: Dict[str, Dict[str, Any]], field: str) -> List[Any]:
    """
    检索给定字段的可能值。

    :param params: 参数字典。
    :param field: 要获取可能值的字段。
    :return: 可能值的列表。
    """

    # 从参数中提取字段信息
    field_info = params["properties"][field]

    # 根据字段的类型或是否存在'enum'，确定并返回可能的值
    if "enum" in field_info:
        return field_info["enum"]
    elif field_info["type"] == "integer":
        return [placeholder_int]
    elif field_info["type"] == "string":
        return [placeholder_string]
    elif field_info["type"] == "boolean":
        return [True, False]
    elif field_info["type"] == "array" and "enum" in field_info["items"]:
        enum_values = field_info["items"]["enum"]
        all_combinations = [
            list(combo)
            for i in range(1, len(enum_values) + 1)
            for combo in itertools.combinations(enum_values, i)
        ]
        return all_combinations
    return []

### 首先为每个函数生成每个调用

提示：

In [16]:
INVOCATION_FILLER_PROMPT = """
1) 在此调用中输入合理的'fill_in_string'和'fill_in_int'值：{invocation}。合理的值由函数定义决定。使用
这里提供的整个函数：{function}，以获取有关适当的fill_in_string和fill_in_int值的上下文。
示例：

输入：invocation: {{
    "name": "control_camera",
    "arguments": {{
      "mode":"video",
      "duration":"fill_in_int"
    }}
}},
function:{function}

输出：invocation: {{
    "name": "control_camera",
    "arguments": {{
      "mode":"video",
      "duration": 30
    }}
}}

确保输出只是一个包含'name'和'arguments'键的字典，没有其他文本或响应。

输入：{invocation}
输出：
"""


COMMAND_GENERATION_PROMPT = """
你需要输出2个命令、问题或陈述，以生成输入的函数和参数,控制的是无人机。
请使命令或问题自然，就像一个人会问的那样，并且命令或问题应该多样化，不要重复。
它不应总是反映函数和参数中使用的精确技术术语，而是反映对话式和直观的请求。
例如，提示不应是'打开圆顶灯'，因为那太技术化了，而应该是'打开里面的灯'。
另一个例子是，提示不应是'打开HVAC'，而是'打开空调'。使用普通驾驶员会使用的语言，即使
它在技术上不正确但在口语中使用。

规则：在撇号或单引号前始终加上反斜杠'\\'。例如，不要说don't，而要说don\\'t。
提示**必须**用双引号。

示例

输入：{{'name': 'calibrate_sensors','arguments': {{}}'' }}
Prompt: ["传感器有问题，你能重置它们吗", "无人机的校准有误，请修复它！"]

输入：{{'name': 'set_autopilot','arguments': {{'status': 'off'}}}}
Prompt: ["好的，我现在想重新控制", "关闭自动驾驶，我准备好控制它了"]

输入：{invocation}
Prompt:
"""

在下面的代码片段中，我们为除`reject_request`函数之外的每个函数生成调用。

为了进行有效的微调，我们需要正确标记的数据。我们可以手动提出示例并标记数据，或者我们可以在`gpt-4o`的帮助下生成合成数据。

经验上，`gpt-4o`需要更多的帮助来获得生成`reject_request`函数的真实示例，所以我们接下来会这样做。

In [39]:
# 工具函数：移除代码块标记并解析JSON
def remove_sequences(input_string):
    """
    移除字符串中的代码块标记，并解析为JSON对象。

    Args:
        input_string (str): 包含代码块的输入字符串。

    Returns:
        dict 或 list: 解析后的JSON对象或列表。

    Raises:
        json.JSONDecodeError: 如果JSON解析失败。
    """
    # 删除"```json"和"```"
    cleaned_string = input_string.replace("```json", "")
    cleaned_string = cleaned_string.replace("```", "")
    cleaned_string = cleaned_string.strip()  # 去除前后空白字符
    return json.loads(cleaned_string)        # 解析为JSON对象或列表

In [64]:
import json

# 工具函数：移除代码块标记并解析JSON
def remove_sequences(input_string):
    """
    移除字符串中的代码块标记，并解析为JSON对象。

    Args:
        input_string (str): 包含代码块的输入字符串。

    Returns:
        dict: 解析后的JSON对象。
    """
    # 删除"```json"和"```"
    cleaned_string = input_string.replace("```json", "").replace("```", "").strip()
    return json.loads(cleaned_string)  # 解析为JSON对象

# 初始化输入对象列表
input_objects = []

# 创建一个列表，包含 function_list 中除了 "reject_request" 之外的所有函数
all_but_reject = [f for f in function_list if f.get("name") != "reject_request"]

for function in all_but_reject:
    # 提取函数名称和参数
    func_name = function["function"]["name"]
    params = function["function"]["parameters"]
    print(f"正在处理函数: {func_name}")

    # 为给定参数生成所有可能的排列
    for arguments in generate_permutations(params):
        print(f"  尝试参数: {arguments}")
        if any(val in arguments.values() for val in ["fill_in_int", "fill_in_str"]):
            # 如果参数包含需要填充的值，则使用模型生成填充的输入对象
            input_object_initial = {"name": func_name, "arguments": arguments}
            messages = [
                {
                    "role": "user",
                    "content": INVOCATION_FILLER_PROMPT.format(
                        invocation=json.dumps(input_object_initial, ensure_ascii=False),
                        function=function
                    ),
                }
            ]
            # 获取模型生成的响应，并解包为 response 和 usage
            response, usage = get_chat_completion(
                model="gpt-4o",
                messages=messages,
                max_tokens=200,
                temperature=0.1
            )
            print(f"    原始生成的输入对象: {response}")

            # 提取 content 并解析为字典
            input_object_parsed = remove_sequences(response.content)
            print(f"    生成的输入对象: {input_object_parsed}")
            input_objects.append(input_object_parsed)
        else:
            # 否则，直接创建输入对象
            input_object = {"name": func_name, "arguments": arguments}
            print(f"    创建的输入对象: {input_object}")
            input_objects.append(input_object)

# 输出生成的输入对象数量
print(f"总共生成了 {len(input_objects)} 个输入对象。")



正在处理函数: takeoff_drone
  尝试参数: {'altitude': 'fill_in_int'}
    原始生成的输入对象: ChatCompletionMessage(content='```json\n{\n    "name": "takeoff_drone",\n    "arguments": {\n        "altitude": 100\n    }\n}\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
    生成的输入对象: {'name': 'takeoff_drone', 'arguments': {'altitude': 100}}
正在处理函数: land_drone
  尝试参数: {'location': 'current'}
    创建的输入对象: {'name': 'land_drone', 'arguments': {'location': 'current'}}
  尝试参数: {'location': 'home_base'}
    创建的输入对象: {'name': 'land_drone', 'arguments': {'location': 'home_base'}}
  尝试参数: {'location': 'custom'}
    创建的输入对象: {'name': 'land_drone', 'arguments': {'location': 'custom'}}
正在处理函数: control_drone_movement
  尝试参数: {'direction': 'forward', 'distance': 'fill_in_int'}
    原始生成的输入对象: ChatCompletionMessage(content='```json\n{\n    "name": "control_drone_movement",\n    "arguments": {\n        "direction": "forward",\n        "distance": 100\n    }\n}\n```', refusal=None, role='

In [65]:
input_objects

[{'name': 'takeoff_drone', 'arguments': {'altitude': 100}},
 {'name': 'land_drone', 'arguments': {'location': 'current'}},
 {'name': 'land_drone', 'arguments': {'location': 'home_base'}},
 {'name': 'land_drone', 'arguments': {'location': 'custom'}},
 {'name': 'control_drone_movement',
  'arguments': {'direction': 'forward', 'distance': 100}},
 {'name': 'control_drone_movement',
  'arguments': {'direction': 'backward', 'distance': 10}},
 {'name': 'control_drone_movement',
  'arguments': {'direction': 'left', 'distance': 10}},
 {'name': 'control_drone_movement',
  'arguments': {'direction': 'right', 'distance': 10}},
 {'name': 'control_drone_movement',
  'arguments': {'direction': 'up', 'distance': 10}},
 {'name': 'control_drone_movement',
  'arguments': {'direction': 'down', 'distance': 10}},
 {'name': 'set_drone_speed', 'arguments': {'speed': 10}},
 {'name': 'control_camera', 'arguments': {'mode': 'photo'}},
 {'name': 'control_camera', 'arguments': {'mode': 'photo', 'duration': 10}},
 

现在我们有了所有的调用，让我们使用`gpt-4o`来生成会产生这些调用的提示。

In [66]:
def create_commands(invocation_list):
    example_list = []
    for i, invocation in enumerate(invocation_list):
        if i < 100:
            print(
                f"\033[34m{np.round(100*i/len(invocation_list),1)}% complete\033[0m")
            if type(invocation) == str or "json" in invocation:
                invocation = remove_sequences(invocation)
            print(invocation)

        # 使用调用字符串格式化提示
        request_prompt = COMMAND_GENERATION_PROMPT.format(
            invocation=invocation)
        messages = [{"role": "user", "content": f"{request_prompt}"}]
        completion, usage = get_chat_completion(messages, temperature=0.8)
        command_dict = {"Input": invocation, "Prompt": completion.content}
        example_list.append(command_dict)
    return example_list

In [67]:
# 仅打印前10行
training_examples_unformatted = create_commands(input_objects)
training_examples_unformatted

0.0% complete
{'name': 'takeoff_drone', 'arguments': {'altitude': 100}}
1.8% complete
{'name': 'land_drone', 'arguments': {'location': 'current'}}
3.5% complete
{'name': 'land_drone', 'arguments': {'location': 'home_base'}}
5.3% complete
{'name': 'land_drone', 'arguments': {'location': 'custom'}}
7.0% complete
{'name': 'control_drone_movement', 'arguments': {'direction': 'forward', 'distance': 100}}
8.8% complete
{'name': 'control_drone_movement', 'arguments': {'direction': 'backward', 'distance': 10}}
10.5% complete
{'name': 'control_drone_movement', 'arguments': {'direction': 'left', 'distance': 10}}
12.3% complete
{'name': 'control_drone_movement', 'arguments': {'direction': 'right', 'distance': 10}}
14.0% complete
{'name': 'control_drone_movement', 'arguments': {'direction': 'up', 'distance': 10}}
15.8% complete
{'name': 'control_drone_movement', 'arguments': {'direction': 'down', 'distance': 10}}
17.5% complete
{'name': 'set_drone_speed', 'arguments': {'speed': 10}}
19.3% complete

[{'Input': {'name': 'takeoff_drone', 'arguments': {'altitude': 100}},
  'Prompt': '["请让无人机起飞到100米的高度", "我想让它升到100米，能帮我吗？"]'},
 {'Input': {'name': 'land_drone', 'arguments': {'location': 'current'}},
  'Prompt': '["把无人机降落到现在的位置", "请让无人机在这里降落"]'},
 {'Input': {'name': 'land_drone', 'arguments': {'location': 'home_base'}},
  'Prompt': '["请把无人机降落在基地", "能不能把它送回家？"]'},
 {'Input': {'name': 'land_drone', 'arguments': {'location': 'custom'}},
  'Prompt': '["请把无人机降落到我指定的地方", "我需要让无人机在特定位置着陆，可以帮我吗？"]'},
 {'Input': {'name': 'control_drone_movement',
   'arguments': {'direction': 'forward', 'distance': 100}},
  'Prompt': '["请让无人机向前飞行100米", "能帮我让它朝前移动一百米吗？"]'},
 {'Input': {'name': 'control_drone_movement',
   'arguments': {'direction': 'backward', 'distance': 10}},
  'Prompt': '["请把无人机往后飞10米", "能不能让它后退10米？"]'},
 {'Input': {'name': 'control_drone_movement',
   'arguments': {'direction': 'left', 'distance': 10}},
  'Prompt': '["请让无人机向左飞行10米", "能不能让它往左移动10米？"]'},
 {'Input': {'name': 'control_drone_movem

现在让我们正确格式化训练示例。有关用于函数调用微调的训练数据格式的更多文档，请参见：https://platform.openai.com/docs/guides/fine-tuning/fine-tuning-examples

In [68]:
def remove_descriptions(function_list):
    for function in function_list:
        func = function["function"]
        if "description" in func:
            del func["description"]

        params = func["parameters"]
        if "properties" in params:
            for param in params["properties"].values():
                if "description" in param:
                    del param["description"]

    return function_list


modified_function_list = remove_descriptions(function_list)
modified_function_list 

[{'type': 'function',
  'function': {'name': 'takeoff_drone',
   'parameters': {'type': 'object',
    'properties': {'altitude': {'type': 'integer'}},
    'required': ['altitude']}}},
 {'type': 'function',
  'function': {'name': 'land_drone',
   'parameters': {'type': 'object',
    'properties': {'location': {'type': 'string',
      'enum': ['current', 'home_base', 'custom']},
     'coordinates': {'type': 'object'}},
    'required': ['location']}}},
 {'type': 'function',
  'function': {'name': 'control_drone_movement',
   'parameters': {'type': 'object',
    'properties': {'direction': {'type': 'string',
      'enum': ['forward', 'backward', 'left', 'right', 'up', 'down']},
     'distance': {'type': 'integer'}},
    'required': ['direction', 'distance']}}},
 {'type': 'function',
  'function': {'name': 'set_drone_speed',
   'parameters': {'type': 'object',
    'properties': {'speed': {'type': 'integer', 'minimum': 0}},
    'required': ['speed']}}},
 {'type': 'function',
  'function': {'

In [69]:
import ast
import json



# 初始化训练示例列表
training_examples = []

# 遍历未格式化的训练示例
for idx, prompt in enumerate(training_examples_unformatted):
    try:
        # 处理 "Input" 字段
        if not isinstance(prompt["Input"], dict):
            # 移除代码块并解析JSON
            prompt["Input"] = remove_sequences(prompt["Input"])
        
        # 确保 "arguments" 字段为JSON字符串
        arguments = prompt["Input"].get("arguments")
        if isinstance(arguments, dict):
            prompt["Input"]["arguments"] = json.dumps(arguments)
        elif isinstance(arguments, str):
            # 可选：验证字符串是否为有效的JSON
            try:
                json.loads(arguments)
            except json.JSONDecodeError:
                print(f"[条目 {idx}] 'arguments' 字段的JSON格式无效，跳过该条目。")
                continue
        else:
            print(f"[条目 {idx}] 'arguments' 字段的类型不支持，跳过该条目。")
            continue

        # 处理 "Prompt" 字段
        if not isinstance(prompt["Prompt"], list):
            # 移除代码块并解析JSON列表
            prompt["Prompt"] = remove_sequences(prompt["Prompt"])
            if not isinstance(prompt["Prompt"], list):
                print(f"[条目 {idx}] 'Prompt' 字段解析后不是列表，跳过该条目。")
                continue

        # 遍历每个用户提示
        for p_idx, p in enumerate(prompt["Prompt"]):
            print(f"[条目 {idx}, 提示 {p_idx}] 用户提示: {p}")
            print(f"[条目 {idx}, 提示 {p_idx}] 输入字典: {prompt['Input']}")

            tool_calls = [
                {
                    "id": "call_id",
                    "type": "function",
                    "function": prompt["Input"]
                }
            ]

            training_examples.append(
                {
                    "messages": [
                        {"role": "system", "content": DRONE_SYSTEM_PROMPT},
                        {"role": "user", "content": p},
                        {"role": "assistant", "tool_calls": tool_calls},
                    ],
                    "parallel_tool_calls": False,
                    "tools": modified_function_list,
                }
            )
    except json.JSONDecodeError as e:
        print(f"[条目 {idx}] JSON解析错误: {e}，跳过该条目。")
        continue
    except Exception as e:
        print(f"[条目 {idx}] 发生意外错误: {e}，跳过该条目。")
        continue

# 输出处理后的训练示例数量
print(f"总共处理了 {len(training_examples)} 个训练示例。")

# 如果需要查看训练示例，可以取消下面的注释
# for example in training_examples:
#     print(example)


[条目 0, 提示 0] 用户提示: 请让无人机起飞到100米的高度
[条目 0, 提示 0] 输入字典: {'name': 'takeoff_drone', 'arguments': '{"altitude": 100}'}
[条目 0, 提示 1] 用户提示: 我想让它升到100米，能帮我吗？
[条目 0, 提示 1] 输入字典: {'name': 'takeoff_drone', 'arguments': '{"altitude": 100}'}
[条目 1, 提示 0] 用户提示: 把无人机降落到现在的位置
[条目 1, 提示 0] 输入字典: {'name': 'land_drone', 'arguments': '{"location": "current"}'}
[条目 1, 提示 1] 用户提示: 请让无人机在这里降落
[条目 1, 提示 1] 输入字典: {'name': 'land_drone', 'arguments': '{"location": "current"}'}
[条目 2, 提示 0] 用户提示: 请把无人机降落在基地
[条目 2, 提示 0] 输入字典: {'name': 'land_drone', 'arguments': '{"location": "home_base"}'}
[条目 2, 提示 1] 用户提示: 能不能把它送回家？
[条目 2, 提示 1] 输入字典: {'name': 'land_drone', 'arguments': '{"location": "home_base"}'}
[条目 3, 提示 0] 用户提示: 请把无人机降落到我指定的地方
[条目 3, 提示 0] 输入字典: {'name': 'land_drone', 'arguments': '{"location": "custom"}'}
[条目 3, 提示 1] 用户提示: 我需要让无人机在特定位置着陆，可以帮我吗？
[条目 3, 提示 1] 输入字典: {'name': 'land_drone', 'arguments': '{"location": "custom"}'}
[条目 4, 提示 0] 用户提示: 请让无人机向前飞行100米
[条目 4, 提示 0] 输入字典: {'name': 'control_drone_movem

In [70]:
training_examples

[{'messages': [{'role': 'system',
    'content': '你是一个控制无人机的智能AI。根据用户的命令或请求，\n调用你的一个函数来完成请求。如果你的可用函数无法完成请求，调用reject_request函数。\n如果请求含糊不清或不明确，拒绝请求。'},
   {'role': 'user', 'content': '请让无人机起飞到100米的高度'},
   {'role': 'assistant',
    'tool_calls': [{'id': 'call_id',
      'type': 'function',
      'function': {'name': 'takeoff_drone',
       'arguments': '{"altitude": 100}'}}]}],
  'parallel_tool_calls': False,
  'tools': [{'type': 'function',
    'function': {'name': 'takeoff_drone',
     'parameters': {'type': 'object',
      'properties': {'altitude': {'type': 'integer'}},
      'required': ['altitude']}}},
   {'type': 'function',
    'function': {'name': 'land_drone',
     'parameters': {'type': 'object',
      'properties': {'location': {'type': 'string',
        'enum': ['current', 'home_base', 'custom']},
       'coordinates': {'type': 'object'}},
      'required': ['location']}}},
   {'type': 'function',
    'function': {'name': 'control_drone_movement',
     'parameters': {'type':

现在，回到拒绝函数。让我们生成一些几乎可行但应导致调用`reject_request`函数的提示。为此，我们查询了`gpt-4o`，要求提供与给定函数列表相关但实际上不可行的请求。

In [71]:
reject_list = [
    "翻译广播消息到另一种语言",
    "在检测到人脸时自动拍照",
    "检测附近的无人机",
    "测量风阻",
    "捕捉慢动作视频",
    "同时向前和向后移动无人机相同的距离",
    "根据地面高度变化调整无人机的高度",
    "在LED显示屏上显示自定义信息",
    "将无人机的时间与智能手机同步",
    "当无人机飞出指定区域时发出警报",
    "校准传感器并同时着陆",
    "检测湿度水平",
    "自动跟随GPS标记的物体",
    "切换夜视模式",
    "在电池电量低时保持当前高度",
    "使用AI决定最佳着陆点",
    "根据风向规划无人机的路线",
]

In [72]:
reject_training_list = []
for prompt in reject_list:
    # 调整格式
    tool_calls = [
        {
            "id": "call_id",
            "type": "function",
            "function": {"name": "reject_request", "arguments": "{}"},
        }
    ]
    reject_training_list.append(
        {
            "messages": [
                {"role": "system", "content": DRONE_SYSTEM_PROMPT},
                {"role": "user", "content": prompt},
                {"role": "assistant", "tool_calls": tool_calls},
            ],
            "parallel_tool_calls": False,
            "tools": modified_function_list,
        }
    )
    
reject_training_list

[{'messages': [{'role': 'system',
    'content': '你是一个控制无人机的智能AI。根据用户的命令或请求，\n调用你的一个函数来完成请求。如果你的可用函数无法完成请求，调用reject_request函数。\n如果请求含糊不清或不明确，拒绝请求。'},
   {'role': 'user', 'content': '翻译广播消息到另一种语言'},
   {'role': 'assistant',
    'tool_calls': [{'id': 'call_id',
      'type': 'function',
      'function': {'name': 'reject_request', 'arguments': '{}'}}]}],
  'parallel_tool_calls': False,
  'tools': [{'type': 'function',
    'function': {'name': 'takeoff_drone',
     'parameters': {'type': 'object',
      'properties': {'altitude': {'type': 'integer'}},
      'required': ['altitude']}}},
   {'type': 'function',
    'function': {'name': 'land_drone',
     'parameters': {'type': 'object',
      'properties': {'location': {'type': 'string',
        'enum': ['current', 'home_base', 'custom']},
       'coordinates': {'type': 'object'}},
      'required': ['location']}}},
   {'type': 'function',
    'function': {'name': 'control_drone_movement',
     'parameters': {'type': 'object',
      'proper

现在将所有训练示例组合在一起

In [73]:
training_list_total = training_examples + reject_training_list

In [74]:
import json

training_file = "data/drone_training.jsonl"

# 打开文件时指定编码为UTF-8，并设置ensure_ascii=False以保持中文字符可见
with open(training_file, "w", encoding="utf-8") as f:
    for item in training_list_total:
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(f"{json_str}\n")

print(f"训练数据已成功写入 {training_file}")


训练数据已成功写入 data/drone_training.jsonl


# 微调

最后，我们可以启动微调作业

In [75]:
# 上传训练文件
file = client.files.create(
    file=open("data/drone_training.jsonl", "rb"),
    purpose="fine-tune",
)
file_id = file.id
print(f"FileID: {file_id}")

# 创建微调作业

ft = client.fine_tuning.jobs.create(
    model="gpt-4o-mini-2024-07-18",#注意Model gpt-4o-mini is not available for fine-tuning，得加上日期
    training_file=file_id,
    suffix="drone",
)

print(f"微调作业已创建: {ft}")

FileID: file-euACikKDXdGo6xuEpNILf44s
微调作业已创建: FineTuningJob(id='ftjob-CMRQnoYCWJRvTMrD03OO4etG', created_at=1730904648, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BbylnC34ErFQAfVanaQGLcaB', result_files=[], seed=1308775555, status='validating_files', trained_tokens=None, training_file='file-euACikKDXdGo6xuEpNILf44s', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='drone')


除了创建微调作业外，您还可以列出现有作业、检索作业的状态或取消作业。

In [76]:
ftjob_id = ft.id
ftjob_id
# 列出10个微调作业
# client.fine_tuning.jobs.list(limit=10)

# 检索微调的状态
# client.fine_tuning.jobs.retrieve(ftjob_id)

# 取消作业
# client.fine_tuning.jobs.cancel("ftjob-abc123")

# 列出微调作业的最多10个事件
# client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10)

# 删除微调模型（必须是创建模型的组织的所有者）
# client.models.delete("ft:gpt-4o-mini:abc:suffix:abc123")

'ftjob-CMRQnoYCWJRvTMrD03OO4etG'

In [77]:
# 检索微调的状态
client.fine_tuning.jobs.retrieve(ftjob_id)

FineTuningJob(id='ftjob-CMRQnoYCWJRvTMrD03OO4etG', created_at=1730904648, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-BbylnC34ErFQAfVanaQGLcaB', result_files=[], seed=1308775555, status='validating_files', trained_tokens=None, training_file='file-euACikKDXdGo6xuEpNILf44s', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='drone')

In [78]:
response = client.fine_tuning.jobs.retrieve(ftjob_id)

print("作业 ID:", response.id)
print("状态:", response.status)
print("已训练的标记数:", response.trained_tokens)

作业 ID: ftjob-CMRQnoYCWJRvTMrD03OO4etG
状态: validating_files
已训练的标记数: None


In [79]:
response = client.fine_tuning.jobs.list_events(ftjob_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-CMRQnoYCWJRvTMrD03OO4etG
Validating training file: file-euACikKDXdGo6xuEpNILf44s


In [81]:
response = client.fine_tuning.jobs.list_events(ftjob_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 378/393: training loss=0.00
Step 379/393: training loss=0.00
Step 380/393: training loss=0.00
Step 381/393: training loss=0.00
Step 382/393: training loss=0.00
Step 383/393: training loss=0.00
Step 384/393: training loss=0.00
Step 385/393: training loss=0.00
Step 386/393: training loss=0.00
Step 387/393: training loss=0.00
Step 388/393: training loss=0.00
Step 389/393: training loss=0.00
Step 390/393: training loss=0.00
Step 391/393: training loss=0.00
Step 392/393: training loss=0.00
Step 393/393: training loss=0.00
Checkpoint created at step 131
Checkpoint created at step 262
New fine-tuned model created
The job has successfully completed


在微调作业完成后，您还可以通过查询微调作业、从result_files中提取文件ID，然后检索该文件的内容，查看训练过程的指标。每个结果CSV文件具有以下列：step、train_loss、train_accuracy、valid_loss和valid_mean_token_accuracy。虽然指标可能有所帮助，但评估来自微调模型的示例可提供最相关的模型质量感受。

In [82]:
fine_tune_results = client.fine_tuning.jobs.retrieve(ftjob_id).result_files
result_file_id = client.files.retrieve(fine_tune_results[0]).id

# 检索结果文件
result_file = client.files.content(file_id=result_file_id)
decoded_content = base64.b64decode(result_file.read()).decode("utf-8")
print(decoded_content)

step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy
1,2.65061,0.80952,,
2,2.97664,0.82353,,
3,2.861,0.82353,,
4,4.40557,0.73333,,
5,3.97439,0.75,,
6,2.05225,0.80952,,
7,1.85767,0.85,,
8,1.28482,0.86364,,
9,1.93404,0.77778,,
10,1.00238,0.86667,,
11,1.79262,0.75,,
12,1.64256,0.71429,,
13,0.12756,0.92857,,
14,0.02119,1.0,,
15,0.01484,1.0,,
16,0.00649,1.0,,
17,0.00178,1.0,,
18,0.00102,1.0,,
19,0.00592,1.0,,
20,9e-05,1.0,,
21,0.0001,1.0,,
22,9e-05,1.0,,
23,6e-05,1.0,,
24,9e-05,1.0,,
25,1.56271,0.875,,
26,0.53574,0.93333,,
27,0.21329,0.95,,
28,6e-05,1.0,,
29,0.00019,1.0,,
30,1.18037,0.875,,
31,0.6001,0.95,,
32,0.22064,0.9375,,
33,0.51295,0.875,,
34,0.00544,1.0,,
35,9e-05,1.0,,
36,7e-05,1.0,,
37,4e-05,1.0,,
38,7e-05,1.0,,
39,0.00014,1.0,,
40,0.05029,0.92857,,
41,0.00038,1.0,,
42,0.00079,1.0,,
43,0.16895,0.875,,
44,0.00055,1.0,,
45,0.01031,1.0,,
46,0.00134,1.0,,
47,0.00053,1.0,,
48,0.03537,1.0,,
49,0.00015,1.0,,
50,0.59634,0.875,,
51,0.00059,1.0,,
52,0.00398,1.0,,
53,0.00036,1.

# 评估

太好了！我们训练了一个用于函数调用的微调模型。让我们看看它在我们的评估集中对无人机助手应自动拒绝的提示的表现。

In [83]:
response = client.fine_tuning.jobs.retrieve(ftjob_id)
ft_model = response.fine_tuned_model
base_model = "gpt-4o-mini"

print(f"\n使用具有挑战性的提示评估微调模型: {ft_model}")
eval(
    model=ft_model,
    function_list=modified_function_list,
    system_prompt=DRONE_SYSTEM_PROMPT,
    prompts_to_expected_tool_name=challenging_prompts_to_expected,
)

print(f"\n使用具有挑战性的提示评估基础模型: {base_model}")
eval(
    model="gpt-4o-mini",
    function_list=function_list,
    system_prompt=DRONE_SYSTEM_PROMPT,
    prompts_to_expected_tool_name=challenging_prompts_to_expected,
)


使用具有挑战性的提示评估微调模型: ft:gpt-4o-mini-2024-07-18:acmr:drone:AQbokhij


匹配数量: 9 / 10 (90.00%)
每次请求的平均延迟: 1285.39 ms
每次请求的平均tokens使用量: 464.80

使用具有挑战性的提示评估基础模型: gpt-4o-mini


匹配数量: 7 / 10 (70.00%)
每次请求的平均延迟: 986.75 ms
每次请求的平均tokens使用量: 463.20


很好！虽然原始模型只拒绝了70%的请求，但微调模型拒绝了100%的请求，并使用了更少的tokens(tools列表中没有了description)。

### 结论

恭喜！您现在已经准备好为函数调用微调您的模型。我们期待看到您构建的东西。